# Análisis de Rendimiento de Operadores de CallMeMaybe 

# Descomposición

__1. Descripción__

El servicio de telefonía virtual CallMeMaybe está desarrollando una nueva función que brindará a los supervisores y las supervisores información sobre los operadores menos eficaces. Se considera que un operador es ineficaz si tiene una gran cantidad de llamadas entrantes perdidas (internas y externas) y un tiempo de espera prolongado para las llamadas entrantes. Además, si se supone que un operador debe realizar llamadas salientes, un número reducido de ellas también será un signo de ineficacia. 

Los datasets contienen información sobre el uso del servicio de telefonía virtual CallMeMaybe. Sus clientes son organizaciones que necesitan distribuir gran cantidad de llamadas entrantes entre varios operadores, o realizar llamadas salientes a través de sus operadores. Los operadores también pueden realizar llamadas internas para comunicarse entre ellos. Estas llamadas se realizan a través de la red de CallMeMaybe.

__2. Objetivo__

Identificar operadores ineficaces para que los supervisores puedan tomar decisiones sobre la formación y gestión del personal a fin de mejorar la eficiencia operativa y la satisfacción del cliente.

__3. Tarea Principal__ 

Identificar operadores ineficaces basados en llamadas perdidas y tiempos de espera prolongados para llamadas entrantes, y baja cantidad de llamadas salientes.

__3.1 Condiciones de Ineficacia:__

Definir las condiciones que hacen a un operador ineficas, según la inormación proporcionada por CallMeMaybe.
- Alto número de llamadas perdidas
- Largo tiempo de espera para atender llamadas entrantes
- Bajo número de llamadas salientes

__3.2 Datos Necesarios:__

- Dataset de estadísticas de llamadas __`telecom_dataset_us.csv`__.
- Dataset de clientes y sus tarifas __`telecom_clients_us.csv`__.

__3.3 Detalles Específicos__

- ¿Cuáles y cuántos son los operadores menos eficaces?
- ¿Cúantas llamadas entrantes pierden los operadores?
- ¿Cuánto demoran en atender las llamadas entrantes?
- ¿Cuántas llamadas salientes realizan los operadores?
- ¿Qué tan eficaces resultan los operadores?
- ¿Cuántos operadores son ineficaces?


__4. Planteamiento de Hipótesis__

- __Hipótesis 1:__ Los operadores ineficaces tienen la misma proporción de llamadas perdidas respecto a la media del total de llamadas perdidas.


- __Hipótesis 2:__ Los operadores poco eficaces demoran en atender llamadas entrantes lo mismo que la media del total de llamadas atendidas.


- __Hipótesis 3:__ Los operadores menos eficaces realizan la misma cantidad de llamadas salientes que la media general de llamadas que salen.

__4.1 Plan de Acción de hipótesis__

__Hipótesis 1:__ 

- Calcular el porcentaje de llamadas perdidas por operador ineficaz.
- Calcular el porcentaje del total de llamadas perdidas.
- Comparar con el promedio y la mediana de ambos grupos mediante gráfico de barras.
- Representar histogramas de distribución en ambos grupos de datos.
- Realizar test de contraste de hipótesis nula y alternativa.

__Hipótesis 2:__ 

- Calcular el tiempo promedio de espera por operador ineficaz.
- Calcular el tiempo promedio de espera por el total de operadores.
- Comparar con el promedio y la mediana de ambos grupos mediante gráfico de barras.
- Representar histogramas de distribución en ambos grupos de datos.
- Realizar test de contraste de hipótesis nula y alternativa.


__Hipótesis 3:__ 

- Calcular el promedio de llamadas salientes por operador ineficaz.
- Calcular el promedio de llamadas salientes por el total de operadores.
- Comparar con el promedio y la mediana de ambos grupos mediante gráfico de barras.
- Representar histogramas de distribución en ambos grupos de datos.
- Realizar test de contraste de hipótesis nula y alternativa.


__5. Preprocesamiento de datos__

- Estudiar y Gestionar valores ausentes.
- Estudiar y Gestionar valores duplicados.
- Estudiar y Gestionar valores outliers.
- Estructurar datasets.
    - _Renombrar columnas._
    - _Estandarizar con snake_case._
    
    
- Estudiar y Gestionar el tipo de datos.
    - _Convertir tipos de datos adecuados._
    
    
- Ingeniería de Características.
    - _Contar número de llamadas perdidas por operador._
    - _Promediar la demora de operadores para atender llamadas entrantes._
    - _Contar número de llamadas salientes de cada operador._
    - _Categorizar eficacia de operadores con valores de 0-1 según condiciones previas. Utilizar operaciones booleanas._


- Contar número de operadores ineficaces y compararlos visualmente con el resto de operadores.
    


__6. Contraste de hipótesis__

Una vez preprocesados los datos e identificadas las hipótesis se procede a evaluarlas para determinar su significancia estadística. Es decir cómo se diferencian las métricas de los operadores poco eficaces de las métricas promedio del total de operadores.

__7. Conclusiones y Recomendaciones__

Mencionar puntos clave y resultados obtenidos, además de proporcionar sugerencias para la gestión de operadores para mejorar la eficiencia y experiencia de los clientes/usuarios.

## Importación de librerías

Se importan las librerías necesarias para el preprocesamiento y EDA

In [2]:
# Importación de librerías
import numpy as np  # Para operaciones numéricas y matrices
import pandas as pd  # Para manipulación de datos en DataFrames
import matplotlib.pyplot as plt  # Para visualización de gráficos
import seaborn as sns  # Para gráficos  más vistosos
from scipy import stats  # Para funciones estadísticas y pruebas 

Matplotlib is building the font cache; this may take a moment.


## Carga y lectura de datos

Se asignan los datasets a variables dataframes

In [7]:
 
calls = pd.read_csv('Proyectos\Telecomunicaciones\notebook\telecom_clients_us.csv', sep=',')
clients = pd.read_csv('telecom_dataset_us', sep=',')



FileNotFoundError: [Errno 2] No such file or directory: 'telecom_dataset_us'

## Fuentes

A continuación se muestra la documentación utilizada para trabajar el proyecto:

- __Tripleten Sprint 3 - Manipulación de datos__
    
    Guía práctica de cómo realizar paso a paso data wrangling. Permite hacer el respectivo preprocesamiento de datos para preparar los datos del proyecto de telecomunicaciones.
    https://tripleten.com/trainer/data-analyst/lesson/66c18ebf-bde5-474f-b9d0-d1b4f668f44f/?from=program


- __Tripleten Sprint 4 - Prueba de hipótesis__

    Guía práctica de cómo realizar una prueba de hipótesis. Permite comparar los operadores ineficaces con el conjunto global de operadores. https://tripleten.com/trainer/data-analyst/lesson/d86b0407-a5aa-407b-83be-6d6094f8b8b4/?from=program
    
    
- __Preprocesamiento de Datos__

    Cómo Preprocesar Datos en Python: Artículo en Built In que ofrece un tutorial paso a paso sobre la implementación del preprocesamiento de datos utilizando Python, NumPy y Pandas. Permite preparar los datos de la compañia de telecomunicaciones para utilizarlos en visualización de resultados y contraste de hipótesis. https://builtin.com/machine-learning/how-to-preprocess-data-python
    

- __Ingeniería de Características__

    Feature Engineering | Python Data Science Handbook: En esta sección del libro Python Data Science Handbook, se cubren ejemplos comunes de tareas de ingeniería de características, como representación de datos categóricos, texto e imágenes, así como la creación de características derivadas para aumentar la complejidad del modelo. La ingeniería de características permite enriquecer el dataset y responder algunas de las preguntas específicas previamente formuladas. https://jakevdp.github.io/PythonDataScienceHandbook/05.04-feature-engineering.html 


- __Graficación en Python__

    Python Data Visualization Tutorial: Video que enseña cómo usar las bibliotecas de visualización de datos en Python, como Matplotlib, Seaborn para crear diferentes tipos de gráficos y visualizaciones útiles para el proyecto de telecomunicaciones. https://www.youtube.com/watch?v=Nt84_TzRkbo
    
    Data Visualization | Data Visualization Python: Video que explica cómo usar Seaborn para crear gráficos estadísticos y visualizar datos. https://www.youtube.com/watch?v=_YWwU-gJI5U
    

In [2]:
print('hello there man')

hello there man


In [1]:
import pandas as pd